In [24]:
import os
source_dir = '/mnt/sda/julie/datasets/pathology/NCT-CRC-HE-100K/NCT-CRC-HE-100K/'
class_names = []
class_maps = {}
class_number = {}

for i, class_dir in enumerate(os.listdir(source_dir)):
    class_images = os.listdir(os.path.join(source_dir, class_dir)) 
    print(class_dir, len(class_images))
    class_number[class_dir] = len(class_images)


BACK 10566
NORM 8763
STR 10446
TUM 14317
ADI 10407
MUC 8896
MUS 13536
LYM 11557
DEB 11512


In [25]:
sorted_class_number = sorted(class_number.items(), key=lambda x:x[1], reverse=True)
sorted_class_number

[('TUM', 14317),
 ('MUS', 13536),
 ('LYM', 11557),
 ('DEB', 11512),
 ('BACK', 10566),
 ('STR', 10446),
 ('ADI', 10407),
 ('MUC', 8896),
 ('NORM', 8763)]

In [26]:
for i, class_name in enumerate(sorted_class_number):
    class_maps[class_name[0]] = i

In [28]:
def generate_pareto_distribution(largest_class_size, num_classes, imbalance_ratio):
    smallest_class_size = largest_class_size / imbalance_ratio
    
    # Calculate the ratio factor for the geometric series
    factor = (largest_class_size / smallest_class_size)**(1 / (num_classes - 1))
    
    class_instances = [round(largest_class_size / (factor**i)) for i in range(num_classes)]
    
    # Ensure the largest class has the exact number of instances
    class_instances[0] = largest_class_size
    
    return class_instances

# Parameters
largest_class_size = 10000
num_classes = 9
imbalance_ratios = [100, 200, 500]

# Generate distributions for different imbalance ratios
distributions = []
for ir in imbalance_ratios:
    distribution = generate_pareto_distribution(largest_class_size, num_classes, ir)
    distributions.append(distribution)
    print(f"Imbalance Ratio {ir}: {distribution} - Total Instances: {sum(distribution)}")
import random
seed = 1
random.seed(seed)
train_samples = [[] for i in range(3)]
val_samples = [[] for i in range(3)]
test_samples = [[] for i in range(3)]

for z in range(len(imbalance_ratios)):
    ir = imbalance_ratios[z]
    distribution = distributions[z]
    for i, class_dir in enumerate(os.listdir(source_dir)):
        class_images = os.listdir(os.path.join(source_dir, class_dir)) 
        random.shuffle(class_images)
        train_s = class_images[:distribution[class_maps[class_dir]]]
        val_s = class_images[distribution[class_maps[class_dir]]:distribution[class_maps[class_dir]]+500]
        test_s = class_images[distribution[class_maps[class_dir]]+500:distribution[class_maps[class_dir]]+1000]
        train_samples[z].extend(['%s/%s' %(class_dir, s) for s in train_s])
        val_samples[z].extend(['%s/%s' %(class_dir, s) for s in val_s])
        test_samples[z].extend(['%s/%s' %(class_dir, s) for s in test_s])


Imbalance Ratio 100: [10000, 5623, 3162, 1778, 1000, 562, 316, 178, 100] - Total Instances: 22719
Imbalance Ratio 200: [10000, 5157, 2659, 1371, 707, 365, 188, 97, 50] - Total Instances: 20594
Imbalance Ratio 500: [10000, 4599, 2115, 972, 447, 206, 95, 43, 20] - Total Instances: 18497


In [31]:
dic = {}
for i, class_dir in enumerate(os.listdir(source_dir)):
    class_images = os.listdir(os.path.join(source_dir, class_dir)) 
    for s in class_images:
        dic['%s/%s' %(class_dir, s)] = class_maps[class_dir]

In [34]:
import numpy as np
for z in range(len(imbalance_ratios)):
    np.save('../numpy/nct/train_%s.npy' %(imbalance_ratios[z]), train_samples[z])
    np.save('../numpy/nct/val_%s.npy' %(imbalance_ratios[z]), val_samples[z])
    np.save('../numpy/nct/test_%s.npy' %(imbalance_ratios[z]), test_samples[z])
np.save('../numpy/nct/dic.npy', dic)

In [35]:
source_dir = '/mnt/sda/julie/projects/OpenMedLongTailed/configs/nct/100/'
for configs in os.listdir(source_dir):
    os.system('mv '+source_dir+configs+' '+source_dir+configs.replace('isic', 'nct'))